In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import backend as K

In [22]:
# dimensions of our images.
img_height, img_width = 200, 200

data_dir = '../input/no-obs-diffs/no_obs_diffs'
epochs = 50
batch_size = 32
reg = l1_l2(0.01)

input_shape = (img_height, img_width, 1)

In [23]:
inp = Input(shape=input_shape)

x = Conv2D(8, (3, 3), padding='same', activation = 'relu', 
           kernel_regularizer=reg, bias_regularizer=reg)(inp)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)

x = Conv2D(16, (5, 5), padding='same', activation = 'relu',
          kernel_regularizer=reg, bias_regularizer=reg)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)

x = Conv2D(32, (5, 5), padding='same', activation = 'relu',
          kernel_regularizer=reg, bias_regularizer=reg)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)

x = Conv2D(64, (5, 5), padding='same', activation = 'relu',
          kernel_regularizer=reg, bias_regularizer=reg)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(4, 4), padding='same')(x)

x = Flatten()(x)
x = Dropout(0.5)(x)
out = Dense(1, activation='sigmoid', kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01))(x)

model = Model(inp, out)
optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 200, 200, 1)]     0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 200, 200, 8)       80        
_________________________________________________________________
batch_normalization_16 (Batc (None, 200, 200, 8)       32        
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 100, 100, 8)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 100, 100, 16)      3216      
_________________________________________________________________
batch_normalization_17 (Batc (None, 100, 100, 16)      64        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 50, 50, 16)       

In [24]:
# this is the augmentation configuration we will use for training
datagen = ImageDataGenerator(
    validation_split=0.3,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

Found 12457 images belonging to 2 classes.
Found 5338 images belonging to 2 classes.


In [25]:
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1)

Epoch 1/50
390/390 [==============================] - 75s 191ms/step - loss: 16.9292 - accuracy: 0.8306 - val_loss: 13.4325 - val_accuracy: 0.5229
Epoch 2/50
390/390 [==============================] - 75s 193ms/step - loss: 9.3002 - accuracy: 0.9144 - val_loss: 6.8172 - val_accuracy: 0.8567
Epoch 3/50
390/390 [==============================] - 74s 191ms/step - loss: 4.8913 - accuracy: 0.9316 - val_loss: 3.7306 - val_accuracy: 0.8837
Epoch 4/50
390/390 [==============================] - 75s 191ms/step - loss: 2.8394 - accuracy: 0.9455 - val_loss: 2.3132 - val_accuracy: 0.8928
Epoch 5/50
390/390 [==============================] - 73s 188ms/step - loss: 1.8465 - accuracy: 0.9462 - val_loss: 1.5672 - val_accuracy: 0.9071
Epoch 6/50
390/390 [==============================] - 74s 189ms/step - loss: 1.2529 - accuracy: 0.9500 - val_loss: 1.2187 - val_accuracy: 0.9176
Epoch 7/50
390/390 [==============================] - 76s 194ms/step - loss: 0.9339 - accuracy: 0.9482 - val_loss: 0.9292 - val_

KeyboardInterrupt: 

## KPI's:
* Basic:                                     av. val. acc. 0.8384788334369659
* With Droput (0.5):                         av. val. acc. 0.8602472841739655
* L1 reg:                                    av. val. acc. 0.7747658252716064
-- Starts to increase stabely -- worth to run more (50+) epochs
* L2 reg av. val. acc. 0.7900711834430695 --more flactuation comparing to l1--
* L1/L2 reg: av. val. acc. 0.7821655988693237 --more flactuation comparing to l1--
* l1, batch 16, lr 0.01 flactuates
* l1, batch 32, lr 0.001 achieves val acc .86 during training, falctuating
* l1, batch 32, lr 0.0001 --BEST SO FAR-- achieves val acc .92 during training, most stable
* l2, batch 32, lr 0.0001 More stable than above, but achieves lower val acc (~.86)
* l1_l2 batch 32, lr 0.0001 Most stable, achieves .89-.90. Try same reg with different params/lr.
* Same with l1_l2(0.001), stable as above, lower val acc

In [ ]:
#model.save_weights('no_obs_2.h5')